# Composing Elements

In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh')

HoloViews supports two primary composition operators that apply to nearly all visual HoloViews objects and create heterogeous collections: ``+`` to generate [``Layout``](../reference/containers/bokeh/Layout.ipynb) objects and ``*`` to create [``Overlay``](../reference/containers/bokeh/Overlay.ipynb) objects.

* **Layout:** A collection of HoloViews objects to be  grouped and displayed side-by-side 
* **Overlay:** A collection of HoloViews objects to be displayed simultaneously, overlaid in the same space.

The Layout and Overlay containers allow you to mix types, and a contained item is not associated with a key. In contrast, [Dimensioned containers](./04-Dimensioned_Containers.ipynb) such as [``HoloMap``](../reference/containers/bokeh/HoloMap.ipynb) , [``GridSpace``](../reference/containers/bokeh/GridSpace.ipynb) and [``NdLayout``](../reference/containers/bokeh/NdLayout.ipynb), do not allow mixed types, and each item has an associated key.

As you can compose a mix of HoloViews elements into layouts and overlays, these types of container are very common, which is why they have dedicated composition operators. This user guide describes how you can build and organize your data using these two types of composition.

To show how layouts and overlays work with heterogeneous types, we will use these two elements throughout this notebook:

In [ ]:
xvals = [0.1* i for i in range(100)]
curve =  hv.Curve((xvals, [np.sin(x) for x in xvals]))
scatter =  hv.Scatter((xvals[::5], np.linspace(0,1,20)))

## ``Layout``

A ``Layout`` can contain any HoloViews object except ``NdLayout``. [Building Composite Objects](05-Building_Composite_Objects.ipynb) contains more details on how to compose containers.

You can build a ``Layout`` from two HoloViews objects (which can have different types) using the ``+`` operator:

In [ ]:
curve + scatter

In this example, we have a ``Layout`` composed of a ``Curve`` element and a ``Scatter`` element. 

### Building ``Layout`` from a list

You can build Layouts of any size by passing a list of objects to the ``Layout`` constructor:

In [ ]:
curve_list   = [hv.Curve((xvals, [np.sin(f*x) for x in xvals])) for f in [0.5, 0.75]]
scatter_list = [hv.Scatter((xvals[::5], f*np.linspace(0,1,20))) for f in [-0.5, 0.5]]
layout = hv.Layout(curve_list + scatter_list).cols(2)
layout

Note the use of the ``.cols`` method to specify the number of columns, wrapping to the next row in scanline order (top-to-bottom and left-to-right).

### A ``Layout`` has two-level attribute access

``Layout`` and ``Overlay`` are fundamentally tree structures holding arbitrary heterogenous HoloViews objects, and are quite different from the dictionary-like dimensioned containers (which will be described in a later guide).

As mentioned previously in [Annotating Data](01-Annotating_Data.ipynb), HoloViews objects have string ``group`` and ``label`` parameters, which allows a two-level attribute access on ``Layout`` objects. First let us see how to index the the above example where the ``group`` and ``label`` parameters were left unspecified:

In [ ]:
layout2 = layout.Curve.I + layout.Scatter.II
layout2

Here we create a second layout by indexing two elements from our earlier ``layout`` object and using ``+`` between them. We see that the first level is the ``group`` string (which defaults to the element class name) followed by the label, which wasn't set and is therefore mapped to a roman numeral (I,II,III, etc). 

As group and label were not specified, our new ``Layout`` will once again have ``Curve.I`` for the curve, but as there is only one scatter element, it will have ``Scatter.I`` to index the scatter:

In [ ]:
layout2.Scatter.I

### Using ``group`` and ``label`` with ``Layout``

Now let's return to the first simple layout example, this time setting a group and label as introduced in the [Annotating Data](../../../user_guide/01-Annotating_Data.ipynb) guide:

In [ ]:
xvals = [0.1* i for i in range(100)]
low_freq =  hv.Curve((xvals, [np.sin(x) for x in xvals]),   group='Sinusoid', label='Low Frequency')
linpoints =  hv.Scatter((xvals[::5], np.linspace(0,1,20)), group='Linear Points', label='Demo')
labelled = low_freq + linpoints
labelled

We can now see how group and label affect access in addition to being used for setting the titles:

In [ ]:
labelled.Linear_Points.Demo + labelled.Sinusoid.Low_Frequency

We have used the semantic group and label names to access the elements and build a new re-ordered layout.

Group and label also allow plot customization; see 
[Customizing Plots](../../../user_guide/03-Customizing_Plots.ipynb) for more information.

## ``Overlay``

An ``Overlay`` can contain any HoloViews elements, but the only container type it can contain is ``NdOverlay``. [Building Composite Objects](05-Building_Composite_Objects.ipynb) provides more detail on how to compose containers.

Other than being composed with ``*`` and displaying elements together in the same space, ``Overlay`` shares many of the same concepts as layout. The rest of this section will show the overlay equivalents of the manipulations shown above for layout. 

First, composition with ``*`` instead of ``+``:

In [ ]:
curve * scatter

### Building ``Overlay`` from a list

An ``Overlay`` can be built explicitly from a list, just like a ``Layout``:

In [ ]:
curve_list   = [hv.Curve((xvals, [np.sin(f*x) for x in xvals])) for f in [0.5, 0.75]]
scatter_list = [hv.Scatter((xvals[::5], f*np.linspace(0,1,20))) for f in [-0.5, 0.5]]
overlay = hv.Overlay(curve_list + scatter_list)
overlay

Note: one concept that applies to ``Overlay`` but not ``Layout`` is that of *color cycles*, which you can learn about in [Customizing Plots](../../../user_guide/03-Customizing_Plots.ipynb).

### ``Overlay`` has two-level attribute access

Like ``Layout``, ``Overlay`` is fundamentally a tree structure holding arbitrary heterogenous HoloViews objects, unlike the dimensioned containers. ``Overlay`` objects also make use of the ``group`` and ``label`` parameters, introduced in [Annotating Data](01-Annotating_Data.ipynb), for two-level attribute access.

Once again, let us see how to index the above example where the ``group`` and ``label`` parameters were left unspecified:

In [ ]:
overlay.Curve.I * overlay.Scatter.II

Here we create a second overlay by indexing two elements from our earlier ``overlay`` object and using ``*`` between them. We see that the first level is the ``group`` string (which defaults to the element class name) followed by the label, which wasn't set and is therefore mapped to a roman numeral (I,II,III etc).

### Using ``group`` and ``label`` with ``Overlay``

Now let's return to the first simple overlay example, this time setting ``group`` and ``label`` as introduced in the [Annotating Data](../../../user_guide/01-Annotating_Data.ipynb) guide:

In [ ]:
high_freq =  hv.Curve((xvals, [np.sin(2*x) for x in xvals]), group='Sinusoid', label='High Frequency')
labelled = low_freq * high_freq * linpoints
labelled

Once again, this example follows the corresponding ``Layout`` example, although this time we added a high-frequency curve to demonstrate how ``group`` and ``label`` are now used to generate the legend (as opposed to the title, as it was for ``Layout``).

The following example shows how ``group`` and ``label`` affect access:

In [ ]:
labelled.Linear_Points.Demo * labelled.Sinusoid.High_Frequency * labelled.Sinusoid.Low_Frequency

This new re-ordered ``Overlay`` switches the z-ordering as well as the legend color of the two sinusoidal curves.

Group and label also allow plot customization; see 
[Customizing Plots](../../../user_guide/03-Customizing_Plots.ipynb) for more information.

## Tab-completion

Both ``Layout`` and ``Overlay`` are designed to be easy to explore and inspect with tab completion. Try running:

```python
overlay.[tab]
```

or

```python
layout.[tab]
```


In a code cell and you should see the first levels of indexing (``Curve`` and ``Scatter``) conveniently listed at the top. If this is not the case, you may need to enable improved tab-completion as described in [Configuring HoloViews](../../../user_guide/Configuring_HoloViews.ipynb).

Having seen how to compose elements, the next section shows how to [customize your plots](../../../user_guide/03-Customizing_Plots.ipynb).